# I210503
# Muhammad Zian Ahmed
# CS-B

# CHROMOSOME BITS DEFINITION

In [1]:
COURSE = 5
TYPE = 1
SECT = 3
STR = 1
PROF = 6
DAY = 2
SLOT = 3
FLOOR = 2
ROOM = 5
RSIZE = 1 
TBITS = COURSE  + TYPE + SECT + STR + PROF + 2*(DAY + SLOT + ROOM + RSIZE + FLOOR)
GENES = (2**SECT) * 5
print(TBITS)

42


# CHROMOSOME GENERATION

In [2]:
from numpy.random import randint
'''
Course, Theory/Lab, Section, Section-Strength, Professor, First-lecture-day,
First- lecture-timeslot, First-lecture-room, First-lecture-room-size, Second-
lecture-day, Second-lecture-timeslot, Second-lecture-room, Second-lecture-
room-size
'''

def random_chromosome():
    return randint(0, 2, (GENES,TBITS))
# print(random_chromosome())

# DECODING GENE FOR EASIER INTERPRETATION

In [3]:
def bintodec(binary):
    decimal = 0
    for digit in binary:
        decimal = decimal*2 + int(digit)
    return decimal
def decode_gene(chromosome):
    t=0
    course = bintodec(chromosome[:COURSE])
    t+=COURSE
    typ = bintodec(chromosome[t:t+TYPE])
    t+=TYPE
    sect = bintodec(chromosome[t:t+SECT])
    t+=SECT
    st = bintodec(chromosome[t:t+STR])
    t+=STR
    prof = bintodec(chromosome[t:t+PROF])
    t+=PROF
    day1 = bintodec(chromosome[t:t+DAY])
    t+=DAY
    slot1 = bintodec(chromosome[t:t+SLOT])
    t+=SLOT
    room1 = bintodec(chromosome[t:t+ROOM])
    t+=ROOM
    rsize1 = bintodec(chromosome[t:t+RSIZE])
    t+=RSIZE
    floor1 = bintodec(chromosome[t:t+FLOOR])
    t+=FLOOR
    day2 = bintodec(chromosome[t:t+DAY])
    t+=DAY
    slot2 = bintodec(chromosome[t:t+SLOT])
    t+=SLOT
    room2 = bintodec(chromosome[t:t+ROOM])
    t+=ROOM
    rsize2 = bintodec(chromosome[t:t+RSIZE])
    t+=RSIZE
    floor2 = bintodec(chromosome[t:t+FLOOR])
    t+=FLOOR
    return course, typ, sect, st, prof, day1, slot1, room1, rsize1,floor1 , day2, slot2, room2, rsize2, floor2


# SCORE CALCULATION BY IDENTIFYING CLASHES

In [4]:
'''
Indexes:
Course0,
Theory/Lab1,
Section2,
Section-Strength3,
Professor4,
      First-lecture-day 5 , 10 Second-lecture-day
First- lecture-timeslot 6 , 11 Second-lecture-timeslot
     First-lecture-room 7 , 12 Second-lecture-room
First-lecture-room-size 8 , 13 Second-lecture-room-size
               floor1   9 , 14 floor2
'''

def fitness(chromosome):
    key1 = {
        1: (6,6),
        2: (6,11),
        3: (11,11)
    }


    score = 0
    decode = [list(decode_gene(chromosome[i])) for i in range(GENES)]
    professor_counts = [0] * (2 ** PROF)
    section_counts = [0] * (2 ** SECT)
    
    for c1 in decode:
        # Count courses per professor/section
        professor_counts[c1[4]] += 1
        section_counts[c1[2]] += 1
        
        if c1[3] != c1[8]: # ROOM SIZE CLASH
            score+=1
        if c1[3] != c1[13]: # ROOM SIZE CLASH
            score+=1

        if c1[1]==0: # THEORY SAME/CLASH DAY CLASH
            if c1[5] == c1[10] or abs(c1[5]-c1[10]) == 1:
                score+=1
        
        if c1[1]==1: #LAB
            if c1[5] != c1[10]: # NON CONSECTIVE LAB
                score+=1
            elif abs(c1[6]-c1[11]) != 1: # NON CONSECTIVE LAB
                score+=1

        for c2 in decode:
            if c1 == c2:
                continue
            key =-1

            # CHECKING IF DAY AND SLOT ARE SAME
            if c1[5] == c2[5] and c1[6]==c2[6]:
                key = 1
            elif c1[5] == c2[10] and c1[6] == c2[11]:
                key = 2
            elif c1[10] == c2[10] and c1[11] == c2[11]:
                key = 3
            if key == -1:
                continue

            i,j=key1[key]
            
            if c1[2] == c2[2]: #SECTION CLASH
                if c1[i+1]!=c2[j+1]: #SAME SECTION DIFF ROOM
                    score+=1

            if c1[4] == c2[4]: # PROFESSOR CLASH
                score+=1

            if c1[i+1] == c2[j+1]: #ROOM-SECTION CLASH
                if c1[2]!=c2[2]:
                    score+=1
                
                
    # Count courses per professor/section
    for count in professor_counts:
        if count > 3:
            score += 1
    for count in section_counts:
        if count > 5:
            score += 1    

    return -score            

# TOURNAMENT SELECTION

In [5]:
from numpy.random import randint
def selection(pop, scores, k=3):
# first random selection
    selection_ix = randint(0,len(pop))
    for ix in randint(0, len(pop), k-1):
    # check if better (e.g. perform a tournament)
        if scores[ix] < scores[selection_ix]:
            selection_ix = ix
    return pop[selection_ix]

# MUTATION
### FLIP THE BITS

In [6]:
from numpy.random import rand
def mutation(bitstring, r_mut):
    for i in range(len(bitstring)):
        for j in range(len(bitstring[i])):
            
            if rand() < r_mut:
                #flip the bits
                if bitstring[i][j]==0:
                    bitstring[i][j]=1
                else:
                    bitstring[i][j]=0


# CROSSOVER
#### SINGLE POINT

In [7]:
from numpy.random import rand

def crossover(p1, p2, r_cross):
    # children are copies of parents by default
    c1, c2 = p1.copy(), p2.copy()
    # check for recombination
    if rand() < 1:
    # select crossover point that is not on the end of the string
        pt = randint(1, len(p1)-2)
        # perform crossover
        c1 = p1[:pt] + p2[pt:]
        c2 = p2[:pt] + p1[pt:]
    
    return [c1, c2]


# PRINTING

In [8]:
import csv

subject = []
prof = []
sect = []
room = []
day = {
    0:'Monday',
    1:'Tuesday',
    2:'Wednesday',
    3:'Thursday',
    4: 'Friday'
    }

for i in range(1, 129):
    subject.append(f'Subject {i}')
    prof.append(f"Professor {i}")
    sect.append(f"Section {i}")
    room.append(f"Room {i}")





'''
Indexes:
Course0,
Theory/Lab1,
Section2,
Section-Strength3,
Professor4,
      First-lecture-day 5 , 10 Second-lecture-day
First- lecture-timeslot 6 , 11 Second-lecture-timeslot
     First-lecture-room 7 , 12 Second-lecture-room
First-lecture-room-size 8 , 13 Second-lecture-room-size
               floor1   9 , 14 floor2
'''


def export_to_csv(timetable):
    headers =   [
                "Course",
                "Theory/Lab",
                "Section",
                "Section Strength",
                "Professor",
                "First Lecture Day",
                "First Lecture Time Slot",
                "First Lecture Room",
                "First Lecture Room Size",
                "Second Lecture Day",
                "Second Lecture Time Slot",
                "Second Lecture Room",
                "Second Lecture Room Size"
                ]

    with open("timetable.csv", mode="w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(headers)
        decoded_timetable = [decode_gene(chromosome) for chromosome in timetable]

        for course_info in decoded_timetable:
                course = f'Subject {course_info[0]+1}'
                theory_lab = "Theory" if course_info[1] == 0 else "Lab"
                section = f'Section {course_info[2]+1}'
                section_strength = "60" if course_info[3] == 0 else "120"
                professor = f'Professor {course_info[4]+1}'
                first_lecture_day = day[course_info[5]]
                first_lecture_time_slot = f"{8 + (course_info[6] - 1) * 1.5} - {9.5 + (course_info[6] - 1) * 1.5}"
                first_lecture_room = f'Room {course_info[7]+1}'
                first_lecture_room_size = "Room" if course_info[8] == 0 else "Hall"
                second_lecture_day = day[course_info[10]]
                second_lecture_time_slot = f"{8 + (course_info[11] - 1) * 1.5} - {9.5 + (course_info[11] - 1) * 1.5}"
                second_lecture_room = f'Room {course_info[12]+1}'
                second_lecture_room_size = "Room" if course_info[13] == 0 else "Hall"
                writer.writerow([course, theory_lab,section, section_strength, professor,
                                 first_lecture_day, first_lecture_time_slot, first_lecture_room,
                                 first_lecture_room_size, second_lecture_day, second_lecture_time_slot,
                                 second_lecture_room, second_lecture_room_size])    

# BEST RESULT CLASHES

In [9]:
'''
Indexes:
Course0,
Theory/Lab1,
Section2,
Section-Strength3,
Professor4,
      First-lecture-day 5 , 10 Second-lecture-day
First- lecture-timeslot 6 , 11 Second-lecture-timeslot
     First-lecture-room 7 , 12 Second-lecture-room
First-lecture-room-size 8 , 13 Second-lecture-room-size
               floor1   9 , 14 floor2
'''

def finalclashes(chromosome):
    key1 = {
        1: (6,6),
        2: (6,11),
        3: (11,11)
    }


    score = 0
    decode = [list(decode_gene(chromosome[i])) for i in range(GENES)]
    professor_counts = [0] * (2 ** PROF)
    section_counts = [0] * (2 ** SECT)
    
    for c1 in decode:
        # Count courses per professor/section
        professor_counts[c1[4]] += 1
        section_counts[c1[2]] += 1
        
        if c1[3] != c1[8] and c1[3]==1: # ROOM SIZE CLASH
            print('Section strength = 120 but class capacity=60')
            score+=1
        if c1[3] != c1[13] and c1[3]==1: # ROOM SIZE CLASH
            print('Section strength = 120 but class capacity=60')
            score+=1

        if c1[1]==0: # THEORY SAME/CLASH DAY CLASH
            if c1[5] == c1[10] or abs(c1[5]-c1[10]) == 1:
                score+=1
        
        if c1[1]==1: #LAB
            if c1[5] != c1[10]: # NON CONSECTIVE LAB
                print(f'Non consective lab for Subject {c1[0]+1}')
                score+=1
            elif abs(c1[6]-c1[11]) != 1: # NON CONSECTIVE LAB
                print(f'Non consective lab for Subject {c1[0]+1}')
                score+=1

        for c2 in decode:
            if c1 == c2:
                continue
            key =-1

            # CHECKING IF DAY AND SLOT ARE SAME
            if c1[5] == c2[5] and c1[6]==c2[6]:
                key = 1
            elif c1[5] == c2[10] and c1[6] == c2[11]:
                key = 2
            elif c1[10] == c2[10] and c1[11] == c2[11]:
                key = 3
            if key == -1:
                continue

            i,j=key1[key]
            
            if c1[2] == c2[2]: #SECTION CLASH
                if c1[i+1]!=c2[j+1]: #SAME SECTION DIFF ROOM
                    print(f'Section {c1[2]} clash for Subject {c1[0]+1} and Subject {c2[0]+1}')
                    score+=1

            if c1[4] == c2[4]: # PROFESSOR CLASH
                print(f'Proffessor {c1[4]} clash for Subject {c1[0]+1} and Subject {c2[0]+1}')
                score+=1

            if c1[i+1] == c2[j+1]: #ROOM-SECTION CLASH
                if c1[2]!=c2[2]:
                    print(f'Room {c1[i+1]} clash for Subject {c1[0]+1} and Subject {c2[0]+1}')
                    score+=1
                
                
    # Count courses per professor/section
    for i,count in enumerate(professor_counts):
        if count > 3:
            print(f'Professor {i+1} has {count} courses.')
            score += 1
    for count in section_counts:
        if count > 5:
            print(f'Section {i+1} has {count} courses.')
            score += 1    

    return -score            
                


# GENETIC ALGORITHIM

In [10]:
from numpy import argmax

MUTATION_RATE = 0.087
CROSSOVER_RATE = 0.9
POPULATION_SIZE = 30
GENERATIONS = 100


def genetic_algorithm(objective, n_bits, n_iter, n_pop, r_cross,r_mut):
    pop = [random_chromosome().tolist() for _ in range(n_pop)]
    wbest = -10000
    wbestgene=pop[0]
    for gen in range(0,n_iter):
        
        scores = [objective(c) for c in pop]
        best = argmax(scores)
        
        if scores[best] > wbest:
            wbestgene = pop[best]
            wbest = scores[best]
        
        
        
        print(f'GENERATION {gen} best Score: ', scores[best])
        #print(*pop[best],sep= '\n')

        selected = [selection(pop, scores) for _ in range(n_pop)]
        children = list()
        for i in range(0, n_pop, 2):
        # get selected parents in pairs
            p1, p2 = selected[i], selected[i+1]
            for c in crossover(p1, p2, r_cross):
                mutation(c, r_mut)
                children.append(c)
        # replace population
        pop = children

    print(f'********Gloabal best Score: {wbest}*******')
    export_to_csv(wbestgene)
    finalclashes(wbestgene)


genetic_algorithm(fitness, TBITS,GENERATIONS, POPULATION_SIZE, CROSSOVER_RATE, MUTATION_RATE)

GENERATION 0 best Score:  -83
GENERATION 1 best Score:  -71
GENERATION 2 best Score:  -77
GENERATION 3 best Score:  -89
GENERATION 4 best Score:  -99
GENERATION 5 best Score:  -92
GENERATION 6 best Score:  -84
GENERATION 7 best Score:  -97
GENERATION 8 best Score:  -86
GENERATION 9 best Score:  -90
GENERATION 10 best Score:  -90
GENERATION 11 best Score:  -108
GENERATION 12 best Score:  -99
GENERATION 13 best Score:  -95
GENERATION 14 best Score:  -98
GENERATION 15 best Score:  -85
GENERATION 16 best Score:  -86
GENERATION 17 best Score:  -90
GENERATION 18 best Score:  -96
GENERATION 19 best Score:  -75
GENERATION 20 best Score:  -88
GENERATION 21 best Score:  -112
GENERATION 22 best Score:  -116
GENERATION 23 best Score:  -76
GENERATION 24 best Score:  -101
GENERATION 25 best Score:  -94
GENERATION 26 best Score:  -105
GENERATION 27 best Score:  -102
GENERATION 28 best Score:  -90
GENERATION 29 best Score:  -113
GENERATION 30 best Score:  -96
GENERATION 31 best Score:  -90
GENERATION 